In [4]:
import os
import re

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import glob


In [30]:
model_dir = 'imagenet'
images_dir = 'images/images_sample/'
folder_list = glob.glob('images/images_sample/*')

In [31]:
% time
image_list = []
for folder in folder_list:
    for image in glob.glob(folder + '/*'):
        image_list.append(image)

Wall time: 0 ns


In [34]:
def create_graph():
    with gfile.FastGFile(os.path.join(
    model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [35]:
def extract_features(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        for ind, image in enumerate(list_images):
            if (ind%100 == 0):
                print('Processing %s...' % (image))
            if not gfile.Exists(image):
                tf.logging.fatal('File does not exist %s', image)

            image_data = gfile.FastGFile(image, 'rb').read()
            predictions = sess.run(next_to_last_tensor,
            {'DecodeJpeg/contents:0': image_data})
            features[ind,:] = np.squeeze(predictions)
            labels.append(re.split('_\d+',image.split('/')[1])[0])

    return features, labels


In [36]:
features,labels = extract_features(image_list)

Processing images/images_sample\6811957\6811957_33d08c8dc440c89bccc8d9889c5485a6.jpg...
Processing images/images_sample\6812009\6812009_aca1ba758a8595ed1521a1141123e3bf.jpg...
Processing images/images_sample\6812091\6812091_0ea9661fe01d44f88b4a91fc337451bb.jpg...


KeyboardInterrupt: 

In [37]:
features

NameError: name 'features' is not defined